# Calling Necessary Libraries AND Getting Today's Date

In [16]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from dateutil import parser
from scipy.stats import poisson
import warnings

warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Get today's date
given_date = "2025-09-23" #year-month-day
threshold = 70 #threshold for percentages to highlight in final dataframe (between 1 and 100)

wanted_leagues = ['austria', 'belgium','denmark', 
                  'england', 'england2', 'england3', 'england4', 'england5', 'france', 'france2',
                  'germany', 'germany2', 'greece', 'italy', 'italy2', 'japan', 'southkorea', 'usa',
                  'netherlands', 'norway','poland', 'portugal', 'portugal2',
                   'saudiarabia','scotland', 'spain', 'spain2', 'sweden', 'switzerland', 'turkey']
# urug, peru, ecuado,

In [17]:
#Calculating Days between given date and today
# Today's date
today = datetime.now().date()

# Specific date
specific_date = datetime.strptime(given_date, "%Y-%m-%d").date()

# Calculate the difference in days
difference = specific_date - today

# Add one day to the difference
days_until_specific_date = difference.days + 1

# Scraping Today's Matches and Leagues

In [18]:
URL = "https://www.soccerstats.com/matches.asp?matchday=" + str(days_until_specific_date) + "&listing=2"
page = requests.get(URL)
liqa = []
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id="btable")
sth = results.find_all("tr", attrs={'height': '34'})

# Converting Date and Collecting Leagues for Analysis

In [19]:
from dateutil import parser

day_abbreviations = {0: "Mon", 1: "Tue", 2: "Wed", 3: "Thu", 4: "Fri", 5: "Sat", 6: "Sun"}
given_date_parsed = parser.parse(given_date)

# Manually format the day to remove leading zeros
day = given_date_parsed.day
month = given_date_parsed.strftime('%b')

# Format the date as "Su 1 Oct" or "Tu 10 Oct" without leading zero for single-digit days
formatted_date = f"{day_abbreviations[given_date_parsed.weekday()]} {day} {month}"
print(formatted_date)

Tue 23 Sep


# Scraping the Web for the League Statistics

In [20]:
final =  pd.DataFrame()
liqa = ''
unique_leagues = wanted_leagues
next_matches = pd.DataFrame()

for i in unique_leagues:
    URL = "https://www.soccerstats.com/results.asp?league=" + i + "&pmtype=bydate"
    page = requests.get(URL)
    liqa = i
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="btable")
    sth = results.find_all("tr", class_="odd")
    sth


    date, league, home, away, ft, ht = [], [], [], [], [],[]
    for i in sth:
        date.append(i.find_all("td", align = 'right')[0].get_text(strip=True))
        league.append(liqa.capitalize())
        home.append(i.find_all("td", align = 'right')[1].get_text(strip=True))
        away.append(i.find("td", align = "left").get_text(strip = True))
        ft.append(i.find_all("td", align = 'center')[0].get_text(strip = True))
        try:
            ht.append(i.find_all("td", align = 'center')[2].get_text(strip = True))
        except IndexError as e:
            ht.append('NA')#print("Last output before error occurred:", i.find_all("td", align = 'center'))

    data = {'Date': date, 'League': league,'Home': home, 'Away': away, 'FT': ft, 'HT': ht}

# Create a DataFrame from the dictionary
    df = pd.DataFrame(data)

# Replace empty strings with NaN
    next_df = df[(df['Date'] == formatted_date) & (df['HT'] == '')]
    next_matches = pd.concat([next_matches, next_df], ignore_index = True)
    df.replace('', pd.NA, inplace=True)

# Drop rows with NaN values
    df_cleaned = df.dropna()

#For Half-Time Results
    hthg, htag = [], []
    for i in df_cleaned['HT']:
        if i == 'NA':
            hthg.append('NA')
            htag.append('NA')
        elif i == '+' or i == '-':
            hthg.append('NA')
            htag.append('NA')
        else:
            try:
                hthg.append(int(i[1]))
                htag.append(int(i[3]))
            except IndexError as e:
                print("Last output before error occurred:", i)



#For Full-Time Results
    hg, ag, tg = [], [], []
    for i in df_cleaned['FT']:
        if len(i) < 5 or ':' in i:
            hg.append('NA')
            ag.append('NA')
            tg.append('NA')
        else:
            try:
                hghg = int(i.split(' - ')[0])
                hg.append(hghg)
                agag = int(i.split(' - ')[1])
                ag.append(agag)
                tg.append(hghg + agag)
            except:
                print(hghg + agag)

    
    df_cleaned['FTHG'], df_cleaned['FTAG'], df_cleaned['FTTG'] = hg, ag, tg
    df_cleaned['HTHG'], df_cleaned['HTAG'] = hthg, htag
    df_cleaned['HTTG'] = df_cleaned['HTHG'] + df_cleaned['HTAG']
    
    final = pd.concat([final, df_cleaned], ignore_index=True)
    
final = final[final['HT'] != 'NA']

# Define current year
current_year = datetime.now().year

# Function to parse and assign the correct year
def parse_date(date_str):
    # Extract day and month, strip any leading/trailing whitespace
    date_part = date_str[3:].strip()
    date_obj = datetime.strptime(date_part, '%d %b')
    # Assign correct year
    full_date = date_obj.replace(year=current_year)
    
    # If the parsed date is already in the future, assign the previous year
    if full_date > datetime.now():
        full_date = full_date.replace(year=current_year - 1)
    
    return full_date

# Apply the function to parse dates
final['Date'] = final['Date'].apply(parse_date)

# Find the latest date for each league
latest_dates = final.groupby('League')['Date'].max().rename('latest_date')

# Merge with the original DataFrame
final = final.merge(latest_dates, on='League')

# Calculate the time difference in days
final['time_diff'] = (final['latest_date'] - final['Date']).dt.days
combined_df = pd.concat([final.head(), final.tail()])
combined_df

,Date,League,Home,Away,FT,HT,FTHG,FTAG,FTTG,HTHG,HTAG,HTTG,latest_date,time_diff
0,2025-08-01,Austria,LASK Linz,Sturm Graz,0 - 2,(0-2),0,2,2,0,2,2,2025-09-21,51
1,2025-08-02,Austria,Tirol,Hartberg,4 - 2,(2-2),4,2,6,2,2,4,2025-09-21,50
2,2025-08-02,Austria,Wolfsberger AC,Altach,0 - 2,(0-0),0,2,2,0,0,0,2025-09-21,50
3,2025-08-02,Austria,Ried,Salzburg,2 - 2,(1-1),2,2,4,1,1,2,2025-09-21,50
4,2025-08-03,Austria,Grazer AK,Austria Wien,2 - 2,(1-1),2,2,4,1,1,2,2025-09-21,49
2689,2025-09-21,Turkey,Basaksehir,Alanyaspor,1 - 1,(1-0),1,1,2,1,0,1,2025-09-22,1
2690,2025-09-21,Turkey,Kocaelispor,Rizespor,1 - 1,(1-1),1,1,2,1,1,2,2025-09-22,1
2691,2025-09-21,Turkey,Kasimpasa,Fenerbahce,1 - 1,(0-1),1,1,2,0,1,1,2025-09-22,1
2692,2025-09-21,Turkey,Samsunspor,F. Karagumruk,3 - 2,(0-1),3,2,5,0,1,1,2025-09-22,1
2693,2025-09-22,Turkey,Galatasaray,Konyaspor,3 - 1,(2-0),3,1,4,2,0,2,2025-09-22,0


In [21]:
next_leagues = next_matches['League'].unique().tolist()
pd.concat([next_matches.head(), next_matches.tail()])

,Date,League,Home,Away,FT,HT
0,Tue 23 Sep,Belgium,Anderlecht,Gent,19:30,
1,Tue 23 Sep,England5,Altrincham,Forest Green,19:45,
2,Tue 23 Sep,England5,Brackley Town,Truro City,19:45,
3,Tue 23 Sep,England5,Gateshead FC,Hartlepool,19:45,
4,Tue 23 Sep,England5,Rochdale,Solihull Moors,19:45,
18,Tue 23 Sep,Scotland,Falkirk,Hibernian,19:45,
19,Tue 23 Sep,Spain,Athletic Bilbao,Girona,18:00,
20,Tue 23 Sep,Spain,Espanyol,Valencia,18:00,
21,Tue 23 Sep,Spain,Levante,Real Madrid,20:30,
22,Tue 23 Sep,Spain,Sevilla FC,Villarreal,20:30,


# Calculating Functions Needed for Dixon-Coles Model

In [22]:
from scipy.optimize import minimize
from scipy.stats import poisson

def rho_correction(x, y, lambda_x, mu_y, rho):
    if x==0 and y==0:
        return 1- (lambda_x * mu_y * rho)
    elif x==0 and y==1:
        return 1 + (lambda_x * rho)
    elif x==1 and y==0:
        return 1 + (mu_y * rho)
    elif x==1 and y==1:
        return 1 - rho
    else:
        return 1.0

def dc_log_like(x, y, alpha_x, beta_x, alpha_y, beta_y, rho, gamma):
    lambda_x, mu_y = np.exp(alpha_x + beta_y + gamma), np.exp(alpha_y + beta_x) 
    return (np.log(rho_correction(x, y, lambda_x, mu_y, rho)) + 
            np.log(poisson.pmf(x, lambda_x)) + np.log(poisson.pmf(y, mu_y)))

def solve_parameters_decay(dataset, half_or_full = 'full', xi=0.001, debug = False, init_vals=None, 
                           options={'disp': True, 'maxiter':100},
                     constraints = [{'type':'eq', 'fun': lambda x: sum(x[:20])-20}] , **kwargs):
    teams = np.sort(dataset['Home'].unique())
    # check for no weirdness in dataset
    away_teams = np.sort(dataset['Away'].unique())
    if not np.array_equal(teams, away_teams):
        raise ValueError("Home Teams Not Equal to Away Teams")
    n_teams = len(teams)
    if init_vals is None:
        # random initialisation of model parameters
        init_vals = np.concatenate((np.random.uniform(0,1,(n_teams)), # attack strength
                                      np.random.uniform(0,-1,(n_teams)), # defence strength
                                      np.array([0,1.0]) # rho (score correction), gamma (home advantage)
                                     ))
        
    def dc_log_like_decay(x, y, alpha_x, beta_x, alpha_y, beta_y, rho, gamma, t, xi=xi):
        lambda_x, mu_y = np.exp(alpha_x + beta_y + gamma), np.exp(alpha_y + beta_x) 
        return  np.exp(-xi*t) * (np.log(rho_correction(x, y, lambda_x, mu_y, rho)) + 
                                  np.log(poisson.pmf(x, lambda_x)) + np.log(poisson.pmf(y, mu_y)))

    def estimate_paramters(params):
        score_coefs = dict(zip(teams, params[:n_teams]))
        defend_coefs = dict(zip(teams, params[n_teams:(2*n_teams)]))
        rho, gamma = params[-2:]
        if half_or_full == 'full':
            log_like = [dc_log_like_decay(row.FTHG, row.FTAG, score_coefs[row.Home], defend_coefs[row.Home],
                                      score_coefs[row.Away], defend_coefs[row.Away], 
                                      rho, gamma, row.time_diff, xi=xi) for row in dataset.itertuples()]
        elif half_or_full == 'half':
            log_like = [dc_log_like_decay(row.HTHG, row.HTAG, score_coefs[row.Home], defend_coefs[row.Home],
                                      score_coefs[row.Away], defend_coefs[row.Away], 
                                      rho, gamma, row.time_diff, xi=xi) for row in dataset.itertuples()]
        return -sum(log_like)
    opt_output = minimize(estimate_paramters, init_vals, options=options, constraints = constraints)
    if debug:
        # sort of hacky way to investigate the output of the optimisation process
        return opt_output
    else:
        return dict(zip(["attack_"+team for team in teams] + 
                        ["defence_"+team for team in teams] +
                        ['rho', 'home_adv'],
                        opt_output.x))

# Calculating Lambda Values for Dixon-Coles Model

In [23]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
stats_df = pd.DataFrame()
full_time_models = []
half_time_models = []

for league in next_leagues:
    league_df = final[final['League'] == league.capitalize()]
    
    full_time_estimates = solve_parameters_decay(league_df, half_or_full = 'full')
    full_time_models.append(full_time_estimates)

    half_time_estimates = solve_parameters_decay(league_df, half_or_full = 'half')
    half_time_models.append(half_time_estimates)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 152.82804035909518
            Iterations: 35
            Function evaluations: 1263
            Gradient evaluations: 35
Optimization terminated successfully    (Exit mode 0)
            Current function value: 98.50261865702782
            Iterations: 46
            Function evaluations: 1638
            Gradient evaluations: 46
Optimization terminated successfully    (Exit mode 0)
            Current function value: 294.2166996683978
            Iterations: 40
            Function evaluations: 2085
            Gradient evaluations: 40
Optimization terminated successfully    (Exit mode 0)
            Current function value: 196.84517288664844
            Iterations: 58
            Function evaluations: 3040
            Gradient evaluations: 58
Optimization terminated successfully    (Exit mode 0)
            Current function value: 129.89791569573765
            Iterations: 48
            Functi

In [24]:
full_time_models[-1]

{'attack_Alaves': 0.46209515000106055,
 'attack_Athletic Bilbao': 0.6300585358046948,
 'attack_Atletico Madrid': 1.3057620166886115,
 'attack_Celta Vigo': 0.4123556033768299,
 'attack_Elche': 1.0115166305640024,
 'attack_Espanyol': 1.5732282497083467,
 'attack_FC Barcelona': 2.031186332060171,
 'attack_Getafe': 1.2037564715722444,
 'attack_Girona': -0.4693074212774599,
 'attack_Levante': 1.714877356014062,
 'attack_Mallorca': 1.1764807302615503,
 'attack_Osasuna': 0.4519913526691854,
 'attack_Rayo Vallecano': 0.8448965807898727,
 'attack_Real Betis': 1.372861937004706,
 'attack_Real Madrid': 1.6371024846732682,
 'attack_Real Oviedo': -0.5488018003095543,
 'attack_Real Sociedad': 0.8666863651669068,
 'attack_Sevilla FC': 1.6945784812959934,
 'attack_Valencia': 1.1744220262633815,
 'attack_Villarreal': 1.4542529176721217,
 'defence_Alaves': -1.8557107777117399,
 'defence_Athletic Bilbao': -1.4951866977287807,
 'defence_Atletico Madrid': -1.6124043176756306,
 'defence_Celta Vigo': -1.1693

# Calculating Probability Matrices for Half/Full Time

In [25]:
#First Function needs work to make it more understandable and a df rather than matrix!
def dixon_coles_simulate_match(params_dict, homeTeam, awayTeam, max_goals=10):
    team_avgs = [np.exp(params_dict['attack_'+homeTeam] + params_dict['defence_'+awayTeam] + params_dict['home_adv']),
                 np.exp(params_dict['defence_'+homeTeam] + params_dict['attack_'+awayTeam])]
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in team_avgs]
    output_matrix = np.outer(np.array(team_pred[0]), np.array(team_pred[1]))
    correction_matrix = np.array([[rho_correction(home_goals, away_goals, team_avgs[0],
                                                   team_avgs[1], params_dict['rho']) for away_goals in range(2)]
                                   for home_goals in range(2)])
    output_matrix[:2,:2] = output_matrix[:2,:2] * correction_matrix
    return output_matrix

full_time_matrices = []
half_time_matrices = []

for i in range(len(next_matches)):
    my_league = next_matches['League'].iloc[i]
    league_index = next_leagues.index(my_league)
    ft_match_score_matrix = dixon_coles_simulate_match(full_time_models[league_index], 
                                                       next_matches['Home'].iloc[i], next_matches['Away'].iloc[i], max_goals = 8)
    ht_match_score_matrix = dixon_coles_simulate_match(half_time_models[league_index], 
                                                       next_matches['Home'].iloc[i], next_matches['Away'].iloc[i], max_goals = 4)
    full_time_matrices.append(ft_match_score_matrix)
    half_time_matrices.append(ht_match_score_matrix)

full_time_matrices[0]

array([[4.60972860e-02, 2.09311762e-02, 3.74056217e-02, 2.18555056e-02,
        9.57736640e-03, 3.35754107e-03, 9.80878734e-04, 2.45619314e-04,
        5.38167867e-05],
       [2.60339768e-02, 1.05504249e-01, 7.34056686e-02, 4.28897564e-02,
        1.87948483e-02, 6.58891729e-03, 1.92489942e-03, 4.82009099e-04,
        1.05611324e-04],
       [4.68847764e-02, 8.21820717e-02, 7.20265023e-02, 4.20839316e-02,
        1.84417254e-02, 6.46512286e-03, 1.88873387e-03, 4.72952977e-04,
        1.03627068e-04],
       [3.06692614e-02, 5.37586746e-02, 4.71154988e-02, 2.75288313e-02,
        1.20634914e-02, 4.22910288e-03, 1.23549854e-03, 3.09378002e-04,
        6.77867293e-05],
       [1.50465193e-02, 2.63743208e-02, 2.31151398e-02, 1.35058059e-02,
        5.91841956e-03, 2.07482265e-03, 6.06142821e-04, 1.51782661e-04,
        3.32565666e-05],
       [5.90552844e-03, 1.03515170e-02, 9.07233841e-03, 5.30082202e-03,
        2.32288905e-03, 8.14336120e-04, 2.37901776e-04, 5.95723702e-05,
        1.3

# Calculating Probabilities of Dixon-Coles Model

In [26]:
ft1, ftx, ft2, ft_score = [], [], [], []
over_15, over_25, under_35, under_45, btts = [], [], [], [], []
ht1, htx, ht2, ht_score, ht_over05, ht_under15 = [], [], [], [], [], []
ho05, ao05, ho15, ao15, hu25, au25 = [], [], [], [], [], []

# Helper function to calculate total goals for each score
def total_goals(i, j):
    return i + j

for i in range(len(next_matches)):
    my_matrix = full_time_matrices[i]
    ht_matrix = half_time_matrices[i]

    ft1.append(round(np.sum(np.tril(my_matrix, k=-1)) * 100, 2)) # Sum of lower triangular values (home win)
    ftx.append(round(np.sum(np.diag(my_matrix)) * 100, 2)) # Sum of diagonal values (draw)
    ft2.append(round(np.sum(np.triu(my_matrix, k=1)) * 100, 2)) # Sum of higher triangular values (away_win)
    
    max_score = np.unravel_index(np.argmax(my_matrix), my_matrix.shape) # Find the index of the maximum score
    home_goals, away_goals = max_score
    ft_score.append(f"{home_goals}-{away_goals}") # Format the score as 'home-away'

    # Calculate the probabilities
    over_15.append(round(np.sum([my_matrix[i, j] for i in range(my_matrix.shape[0]) for j in range(my_matrix.shape[1]) if total_goals(i, j) > 1.5]) * 100, 2))
    over_25.append(round(np.sum([my_matrix[i, j] for i in range(my_matrix.shape[0]) for j in range(my_matrix.shape[1]) if total_goals(i, j) > 2.5]) * 100, 2))
    under_35.append(round(np.sum([my_matrix[i, j] for i in range(my_matrix.shape[0]) for j in range(my_matrix.shape[1]) if total_goals(i, j) <= 3.5]) * 100, 2))
    under_45.append(round(np.sum([my_matrix[i, j] for i in range(my_matrix.shape[0]) for j in range(my_matrix.shape[1]) if total_goals(i, j) <= 4.5]) * 100, 2))

    # Calculate BTTS (both teams to score and goals != 0)
    btts.append(round(np.sum([my_matrix[i, j] for i in range(1, my_matrix.shape[0]) for j in range(1, my_matrix.shape[1])]) * 100, 2)) 

    # Calculate statistics for Half Time
    ht1.append(round(np.sum(np.tril(ht_matrix, k=-1)) * 100, 2)) # Sum of lower triangular values (home win)
    htx.append(round(np.sum(np.diag(ht_matrix)) * 100, 2)) # Sum of diagonal values (draw)
    ht2.append(round(np.sum(np.triu(ht_matrix, k=1)) * 100, 2)) # Sum of higher triangular values (away_win)

    ht_max_score = np.unravel_index(np.argmax(ht_matrix), ht_matrix.shape) # Find the index of the maximum score
    ht_hogs, ht_awgs = ht_max_score
    ht_score.append(f"{ht_hogs}-{ht_awgs}") # Format the score as 'home-away'

    ht_over05.append(round(np.sum([ht_matrix[i, j] for i in range(ht_matrix.shape[0]) for j in range(ht_matrix.shape[1]) if total_goals(i, j) > 0.5]) * 100, 2))   
    ht_under15.append(round(np.sum([ht_matrix[i, j] for i in range(ht_matrix.shape[0]) for j in range(ht_matrix.shape[1]) if total_goals(i, j) < 1.5]) * 100, 2)) 

    ho05.append(round(np.sum(my_matrix[1:,:]) * 100, 2))
    ao05.append(round(np.sum(my_matrix[:,1:]) * 100, 2))
    ho15.append(round(np.sum(my_matrix[2:,:]) * 100, 2))
    ao15.append(round(np.sum(my_matrix[:,2:]) * 100, 2))
    hu25.append(round(np.sum(my_matrix[:3,:]) * 100, 2))
    au25.append(round(np.sum(my_matrix[:,:3]) * 100, 2))
    

# Combine lists into a DataFrame
final_results = pd.DataFrame({
    'League': next_matches['League'], 'Home': next_matches['Home'], 'Away': next_matches['Away'],
    'FT1': ft1, 'FTX': ftx, 'FT2': ft2, 'FTR': ft_score,
    'DC1X': [x + y for x, y in zip(ft1, ftx)], 'DC12': [x + y for x, y in zip(ft1, ft2)], 'DCX2': [x + y for x, y in zip(ftx, ft2)],
    '1.5O': over_15, '2.5O': over_25, '3.5U': under_35, '4.5U': under_45, 'BTTS': btts,
    'HT1': ht1, 'HTX': htx, 'HT2': ht2, 'HTR': ht_score,
    'HTDC1X': [x + y for x, y in zip(ht1, htx)], 'HTDC12': [x + y for x, y in zip(ht1, ht2)], 'HTDCX2': [x + y for x, y in zip(htx, ht2)],
    'HT0.5O': ht_over05, 'HT1.5U': ht_under15, 'H0.5O':ho05, 'A0.5O':ao05, 'H1.5O':ho15, 'A1.5O':ao15, 'H2.5U':hu25, 'A2.5U':au25
})

# Function to highlight values higher than threshold
def highlight_values(value):
    if isinstance(value, str):
        return ''  # Return empty string for NaN values
    elif value > threshold:
    #color = 'red'
        return 'background-color: red'
    else:
        return ''

# Apply the style
with pd.option_context('display.precision', 2):
    styled_df = final_results.style.applymap(highlight_values)
styled_df.to_excel(given_date + ".xlsx", index = False)
# Display the styled DataFrame
from IPython.display import display, HTML
display(styled_df)

,League,Home,Away,FT1,FTX,FT2,FTR,DC1X,DC12,DCX2,1.5O,2.5O,3.5U,4.5U,BTTS,HT1,HTX,HT2,HTR,HTDC1X,HTDC12,HTDCX2,HT0.5O,HT1.5U,H0.5O,A0.5O,H1.5O,A1.5O,H2.5U,A2.5U
0,Belgium,Anderlecht,Gent,41.260000,25.800000,32.920000,1-1,67.060000,74.180000,58.720000,90.660000,71.680000,49.100000,68.430000,73.210000,26.090000,63.790000,10.120000,0-0,89.880000,36.210000,73.910000,38.810000,91.900000,85.920000,82.650000,58.350000,52.280000,68.680000,74.310000
1,England5,Altrincham,Forest Green,12.590000,26.770000,60.640000,0-1,39.360000,73.230000,87.410000,63.540000,35.740000,83.360000,93.520000,35.130000,24.000000,40.860000,34.900000,0-0,64.860000,58.900000,75.760000,75.310000,53.450000,43.480000,78.890000,11.230000,46.060000,97.970000,79.470000
2,England5,Brackley Town,Truro City,64.280000,27.410000,8.310000,1-0,91.690000,72.590000,35.720000,54.710000,27.230000,88.950000,96.280000,23.820000,32.460000,61.730000,5.800000,0-0,94.190000,38.260000,67.530000,41.320000,89.250000,76.560000,30.240000,42.560000,5.120000,82.100000,99.400000
3,England5,Gateshead FC,Hartlepool,9.960000,27.650000,62.390000,0-1,37.610000,72.350000,90.040000,57.080000,29.360000,87.640000,95.670000,27.540000,7.550000,29.090000,61.690000,0-1,36.640000,69.240000,90.780000,80.630000,44.080000,34.890000,76.800000,6.950000,42.900000,99.040000,81.860000
4,England5,Rochdale,Solihull Moors,79.790000,15.930000,4.230000,2-0,95.720000,84.020000,20.160000,73.310000,47.620000,74.020000,88.010000,27.490000,54.860000,39.660000,5.180000,0-0,94.520000,60.040000,44.840000,66.110000,68.340000,89.080000,30.310000,64.960000,5.150000,61.750000,99.350000
5,England5,Woking,Sutton Utd,43.350000,25.050000,31.590000,1-1,68.400000,74.940000,56.640000,83.070000,61.010000,61.300000,78.890000,63.360000,14.610000,45.540000,39.740000,0-0,60.150000,54.350000,85.280000,65.520000,67.730000,81.970000,76.280000,51.090000,42.160000,75.360000,82.390000
6,England5,York City,Carlisle Utd,29.470000,31.850000,38.680000,1-1,61.320000,68.150000,70.530000,63.220000,35.170000,83.770000,93.740000,43.140000,9.530000,64.080000,26.380000,0-0,73.610000,35.910000,90.460000,39.900000,89.790000,61.810000,68.100000,25.050000,31.650000,92.650000,89.170000
7,France2,Amiens,Saint-Etienne,64.960000,14.620000,19.980000,3-1,79.580000,84.940000,34.600000,94.100000,85.130000,29.700000,47.960000,77.260000,63.590000,15.930000,17.050000,1-0,79.520000,80.640000,32.980000,91.170000,33.500000,94.890000,81.700000,80.570000,51.010000,40.910000,74.910000
8,France2,Annecy,Nancy,49.570000,26.260000,24.170000,1-0,75.830000,73.740000,50.430000,57.060000,31.750000,86.100000,94.930000,35.470000,29.440000,51.460000,19.090000,0-0,80.900000,48.530000,70.550000,52.730000,86.020000,70.870000,52.520000,34.940000,17.150000,87.210000,96.020000
9,France2,Guingamp,USL Dunkerque,36.780000,15.550000,46.520000,3-3,52.330000,83.300000,62.070000,97.350000,94.100000,12.160000,23.940000,90.420000,26.700000,11.530000,15.260000,3-2,38.230000,41.960000,26.790000,53.790000,1.270000,94.000000,95.340000,79.360000,83.600000,41.600000,34.910000
